In [1]:
import matplotlib.pyplot as plt
import pandas as pd
import matplotlib.colors as mc
import colorsys

In [27]:
def adjust_lightness(color, amount=0.5):
    try:
        c = mc.cnames[color]
    except:
        c = color
    c = colorsys.rgb_to_hls(*mc.to_rgb(c))
    return colorsys.hls_to_rgb(c[0], max(0, min(1, amount * c[1])), c[2])

def make_plots(csv, task_name, df_modifier = lambda x:x, xlog=False, ylog=False, title="", program_names=["./task1.py", "./task2.py"]):
    df = pd.read_csv(f"{csv}.csv", sep=';')
    df = df.groupby(['program', 'thread_num', 'stars_count'], as_index=False).agg({'time':['mean','std']})
    df = df_modifier(df)
    colors = {
        "./task1.py": "#0fb9b1",
        "./task2.py": "#a55eea",
        "./task3_zeus.py": "#fc5c65",
    }
    error_colors = {key: adjust_lightness(colors[key], 0.5) for key in colors}

    for program in program_names:
        df_ = df[(df["program"]==program)]
        plt.plot(df_["thread_num"], df_["time"]["mean"], "o--", color=colors[program], label=f"Program {program}")
        plt.errorbar(df_["thread_num"], df_["time"]["mean"], df_["time"]["std"], color=error_colors[program], linestyle='None', zorder=3, elinewidth=1, capsize=3)

    plt.title(title)
    plt.xlabel("Liczba procesów")
    plt.ylabel("Czas wykonania [s]")
    if xlog:
        plt.xscale("log")
    if ylog:
        plt.yscale("log")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"images/{task_name}_plot1.jpg", dpi=300)
    plt.clf()

    for program in program_names:
        df_ = df[(df["program"]==program)]
        plt.plot(df_["thread_num"], df_["time"]["mean"].values[0] / df_["time"]["mean"], "o--", color=colors[program], label=f"Program {program}")

    plt.plot(df["thread_num"], df["thread_num"], "-", color="#FF6666", label=f"Idealne przyspieszenie")

    plt.title(title)
    plt.xlabel("Liczba procesów")
    plt.ylabel("Przyspieszenie")
    if xlog:
        plt.xscale("log")
    if ylog:
        plt.yscale("log")
    plt.legend()
    plt.tight_layout()
    plt.savefig(f"images/{task_name}_plot2.jpg", dpi=300)
    plt.clf()

In [28]:
make_plots("report-acer", "plots-acer", lambda df: df[(df["thread_num"]<=8)], title="Zależność przyspieszenia od liczby procesów\nIntel Core i5-7300HQ")
make_plots("report-acer", "plots-acer-threads", xlog=True, ylog=True, title="Zależność przyspieszenia od liczby procesów\nIntel Core i5-7300HQ")
make_plots("report-zeus", "plots-zeus", title="Zależność przyspieszenia od liczby procesów\nZEUS", program_names=["./task1.py", "./task2.py", "./task3_zeus.py"])

<Figure size 432x288 with 0 Axes>